# 케라스 창시자에게 배우는 딥러닝 Main Quest
---
## U-Net 이미지 세그멘테이션
---
### Motorcycle Night Ride 데이터 불러오기

In [ ]:
!pip install kaggle

In [7]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d sadhliroomyprime/motorcycle-night-ride-semantic-segmentation

In [ ]:
!unzip motorcycle-night-ride-semantic-segmentation.zip

In [10]:
import os
import shutil

source_dir = '/content/MotorcycleNightRideDataset/images'

for file in os.listdir(source_dir):
    if file.endswith('.png'):
        if 'fuse' in file:
            shutil.move(os.path.join(source_dir, file), '/content/MotorcycleNightRideDataset/annotations')
        elif 'save' in file:
            shutil.move(os.path.join(source_dir, file), '/content/MotorcycleNightRideDataset/save')

### 데이터 전처리
#### Augmentation 적용 및 정규화 텐서 반환 함수 및 시각화 확인

In [11]:
input_dir = "/content/MotorcycleNightRideDataset/images"
target_dir = "/content/MotorcycleNightRideDataset/annotations"

input_img_paths = sorted(
    [os.path.join(input_dir, fname)
     for fname in os.listdir(input_dir)
     if fname.endswith(".png")])
target_paths = sorted(
    [os.path.join(target_dir, fname)
     for fname in os.listdir(target_dir)
     if fname.endswith(".png") and not fname.startswith(".")])

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.utils import load_img, img_to_array

plt.axis("off")
plt.imshow(load_img(input_img_paths[87]))

In [ ]:
def display_target(target_array):
    normalized_array = (target_array.astype("uint8") - 1) * 50
    plt.axis("off")
    plt.imshow(normalized_array[:, :, 0])

img = img_to_array(load_img(target_paths[87], color_mode="grayscale"))
display_target(img)

In [ ]:
print(len(input_img_paths))
print(len(target_paths))

In [ ]:
from PIL import Image
image_size=Image.open(target_paths[87])
print(image_size)

In [16]:
import numpy as np
import random

img_size = (256, 256)
num_imgs = len(input_img_paths)

random.Random(1337).shuffle(input_img_paths)
random.Random(1337).shuffle(target_paths)

def path_to_input_image(path):
    return img_to_array(load_img(path, target_size=img_size))

def path_to_target(path):
    img = img_to_array(
        load_img(path, target_size=img_size, color_mode="grayscale"))
    img = img.astype("uint8") - 1
    return img

input_imgs = np.zeros((num_imgs,) + img_size + (3,), dtype="float32")
targets = np.zeros((num_imgs,) + img_size + (1,), dtype="uint8")
for i in range(num_imgs):
    input_imgs[i] = path_to_input_image(input_img_paths[i])
    targets[i] = path_to_target(target_paths[i])

num_val_samples = int(0.2 * num_imgs)
train_input_imgs = input_imgs[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_input_imgs = input_imgs[-num_val_samples:]
val_targets = targets[-num_val_samples:]

In [ ]:
print(train_input_imgs.shape)
print(train_targets.shape)
print(val_input_imgs.shape)
print(val_targets.shape)

---
### 모델 생성

In [28]:
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model

def conv_block(input_tensor, num_filters):
    encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
    encoder = layers.BatchNormalization()(encoder)
    encoder = layers.Activation('relu')(encoder)
    encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
    encoder = layers.BatchNormalization()(encoder)
    encoder = layers.Activation('relu')(encoder)
    return encoder

def encoder_block(input_tensor, num_filters):
    encoder = conv_block(input_tensor, num_filters)
    encoder_pool = layers.MaxPooling2D((2, 2), strides=(2,2))(encoder)
    return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
    decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
    decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
    decoder = layers.BatchNormalization()(decoder)
    decoder = layers.Activation('relu')(decoder)
    decoder = conv_block(decoder, num_filters)
    return decoder

def unet_model(input_size, num_classes):
    inputs = Input(shape=input_size + (3,))

    encoder0_pool, encoder0 = encoder_block(inputs, 32)
    encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64)
    encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128)
    encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256)
    encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512)

    center = conv_block(encoder4_pool, 1024)

    decoder4 = decoder_block(center, encoder4, 512)
    decoder3 = decoder_block(decoder4, encoder3, 256)
    decoder2 = decoder_block(decoder3, encoder2, 128)
    decoder1 = decoder_block(decoder2, encoder1, 64)
    decoder0 = decoder_block(decoder1, encoder0, 32)

    outputs = layers.Conv2D(num_classes, (1, 1), activation='softmax')(decoder0)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

# Now create the U-Net model
model = unet_model((512, 512), num_classes=6)
model.summary()


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv2d_60 (Conv2D)          (None, 512, 512, 32)         896       ['input_5[0][0]']             
                                                                                                  
 batch_normalization_27 (Ba  (None, 512, 512, 32)         128       ['conv2d_60[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_27 (Activation)  (None, 512, 512, 32)         0         ['batch_normalization_27

### 컴파일 및 학습

In [27]:
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy")

callbacks = [
    keras.callbacks.ModelCheckpoint("segmentation_motor.h5",
                                    save_best_only=True)
]

history = model.fit(train_input_imgs, train_targets,
                    epochs=10,
                    callbacks=callbacks,
                    batch_size=64,
                    validation_data=(val_input_imgs, val_targets))

Epoch 1/10
3/3 [==============================] - 42s 9s/step - loss: nan - val_loss: nan
Epoch 2/10
3/3 [==============================] - 2s 731ms/step - loss: nan - val_loss: nan
Epoch 3/10
3/3 [==============================] - 2s 750ms/step - loss: nan - val_loss: nan
Epoch 4/10
3/3 [==============================] - 2s 743ms/step - loss: nan - val_loss: nan
Epoch 5/10
3/3 [==============================] - 2s 747ms/step - loss: nan - val_loss: nan
Epoch 6/10
3/3 [==============================] - 2s 751ms/step - loss: nan - val_loss: nan
Epoch 7/10
3/3 [==============================] - 2s 738ms/step - loss: nan - val_loss: nan
Epoch 8/10
3/3 [==============================] - 2s 748ms/step - loss: nan - val_loss: nan
Epoch 9/10
3/3 [==============================] - 2s 745ms/step - loss: nan - val_loss: nan
Epoch 10/10
3/3 [==============================] - 2s 735ms/step - loss: nan - val_loss: nan
